# Model Evaluation

Model evaluation is done by splitting the available data into three sets: training data, validation data and test data. Training data is used to train the model; validation data is used to find the best configuration parameters (hyperparameters) of the model; and the test data is used to evaluate the generalizability of the model.

We consider three classic evaluation recipes: 

* simple hold-out validation
* $K$-fold validation, and 
* iterated $K$-fold validation with shuffling.

## Simple Hold-Out Validation

In [1]:
import numpy as np
from typing import List, Tuple


def train_val_split(data: np.array, val_set_frac: float) -> Tuple[np.array, np.array]:
    assert val_set_frac >= 0 and val_set_frac <= 1
    
    local_cpy = np.copy(data)
    np.random.shuffle(local_cpy)
    
    idx = int(len(local_cpy) * val_set_frac)
    val_data = local_cpy[: idx]
    train_data = local_cpy[idx :]
    
    return train_data, val_data

One tests a set of models with different configurations (hyperparameters) and chooses the best configuration.

```python
scores = []
for model_config in list_of_configurations:
    model = get_model(model_config)
    train, val = train_val_split(training_data)
    model.fit(train)
    scores.append(model.evaluate(val))
```

Then the chosen model is evaluated with the test data.
```python
model = get_model(chosen_config)                                        
model.train(training_data)             
test_score = model.evaluate(test_data) 
```

## $K$-Fold Cross Validation

Partition the dataset into $K$ sets of roughly equal size. For $1 \leq i \leq K$, do the following: train on $K - 1$ partitions and evaluate on the $i$th partition and keep a track of the score. The final score of the model is the average of the scores in each of these iterations. 

In [4]:
from typing import Dict, Any


def k_fold_crossvalidation(data: np.array, k: int, model_configuration: Dict[Any, Any]) -> float:
    assert k > 1
    
    num_validation_samples = len(data) // k

    np.random.shuffle(data)

    validation_scores = []
    
    for fold in range(k):
        validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)] 
        training_data = np.concatenate(data[:num_validation_samples * fold], 
                                       data[num_validation_samples * (fold + 1):], axis=0)
        model = get_model(model_configuration)
        model.train(training_data)
        validation_score = model.evaluate(validation_data)
        validation_scores.append(validation_score)

    return np.average(validation_scores)  

One we have evaluated models with different configurations, we select the configuration that is best and then evaluate the model on the test data.

```python
model = get_model(final_configuration)                                                   
model.train(data)                                                     
test_score = model.evaluate(test_data)  
```